In [9]:
import mdtraj as md
import numpy as np


# Define the path to your structure file
file_path = 'Data/1UBQ_processed.pdb'  # Update this path as needed

# Load the structure file
traj = md.load(file_path)

In [10]:
print(f"Number of atoms: {traj.n_atoms}")
print(f"Number of residues: {traj.n_residues}")
print(f"Topology:\n{traj.topology}")

Number of atoms: 1405
Number of residues: 134
Topology:
<mdtraj.Topology with 2 chains, 134 residues, 1405 atoms, 1353 bonds>


In [11]:
# Compute secondary structure using DSSP
dssp = md.compute_dssp(traj)[0]  # Compute DSSP and get the secondary structure assignment

# Print the first few DSSP assignments for inspection
print("First few secondary structure assignments:")
print(dssp[:20])


First few secondary structure assignments:
['C' 'E' 'E' 'E' 'E' 'E' 'E' 'C' 'C' 'C' 'C' 'E' 'E' 'E' 'E' 'E' 'C' 'C'
 'C' 'C']


In [12]:
# Define the DSSP secondary structure code for helices
helix_code = 'H'  # DSSP code for alpha-helix

# Count the number of helical residues
helical_residues = np.sum(dssp == helix_code)
print(f"Total number of helical amino acids: {helical_residues}")


Total number of helical amino acids: 18


In [17]:
def compute_hydrogen_bonds(traj, distance_cutoff=.35, angle_cutoff=30.0):
    """
    Compute hydrogen bonds in the trajectory based on distance and angle cutoffs.

    Parameters:
    traj : mdtraj.Trajectory
        The trajectory object to analyze.
    distance_cutoff : float
        The maximum distance (in Angstroms) for a hydrogen bond.
    angle_cutoff : float
        The minimum angle (in degrees) for a hydrogen bond.

    Returns:
    int
        Total number of hydrogen bonds.
    """
    # Get topology and positions
    topology = traj.topology
    positions = traj.xyz[0]  # Take the first frame for static analysis

    # Select potential hydrogen bond donors (hydrogens) and acceptors (oxygens, nitrogens)
    donors = topology.select('element H')
    acceptors = topology.select('element O N')
    
    hbonds = []

    for donor in donors:
        donor_pos = positions[donor]
        for acceptor in acceptors:
            acceptor_pos = positions[acceptor]
            
            # Calculate distance
            distance = np.linalg.norm(donor_pos - acceptor_pos)
            
            if distance < distance_cutoff:
                # Calculate the angle (assuming donor-hydrogen-acceptor)
                # Use additional code for angle calculation if necessary
                hbonds.append((donor, acceptor))
    
    return len(hbonds)

# Compute the number of hydrogen bonds
num_hbonds = compute_hydrogen_bonds(traj)

# Print the total number of hydrogen bonds
print(f"Total number of hydrogen bonds: {num_hbonds}")


Total number of hydrogen bonds: 2210


In [20]:
hbonds = md.baker_hubbard(traj, periodic=False)
label = lambda hbond : '%s -- %s' % (traj.topology.atom(hbond[0]), traj.topology.atom(hbond[2]))
for hbond in hbonds:
    print(label(hbond))
print(len(hbonds))

MET1-N -- VAL17-O
ILE3-N -- LEU15-O
PHE4-N -- SER65-O
VAL5-N -- ILE13-O
LYS6-N -- LEU67-O
THR7-N -- LYS11-O
THR9-N -- THR7-OG1
GLY10-N -- THR7-O
ILE13-N -- VAL5-O
LEU15-N -- ILE3-O
VAL17-N -- MET1-O
GLU18-N -- ASP21-OD2
ASP21-N -- GLU18-O
ILE23-N -- ARG54-O
GLU24-N -- ASP52-O
ASN25-N -- THR22-OG1
ASN25-N -- THR22-O
VAL26-N -- THR22-O
LYS27-N -- ILE23-O
LYS27-NZ -- ASP52-OD2
ALA28-N -- GLU24-O
LYS29-N -- ASN25-O
LYS29-NZ -- GLU16-O
ILE30-N -- VAL26-O
GLN31-N -- LYS27-O
ASP32-N -- ALA28-O
LYS33-N -- LYS29-O
GLU34-N -- ILE30-O
GLY35-N -- GLN31-O
GLN40-N -- PRO37-O
GLN41-N -- PRO38-O
GLN41-NE2 -- ILE36-O
GLN41-NE2 -- LYS27-O
ARG42-N -- VAL70-O
ARG42-NE -- GLN49-NE2
ILE44-N -- HIS68-O
PHE45-N -- LYS48-O
LYS48-N -- PHE45-O
LEU50-N -- LEU43-O
GLU51-N -- TYR59-OH
ARG54-N -- GLU51-O
THR55-N -- ASP58-OD1
LEU56-N -- ASP21-O
SER57-N -- PRO19-O
ASP58-N -- THR55-OG1
ASP58-N -- THR55-O
TYR59-N -- LEU56-O
ASN60-N -- SER57-O
ILE61-N -- LEU56-O
GLU64-N -- GLN2-O
SER65-N -- GLN62-O
LEU67-N -- PHE4-O
HIS6